<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2. Решение
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на StackOverflow

### Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 50 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась меньше 5 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на 45 мин.-час)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://cloud.mail.ru/public/3bwi/bFYHDN5S5) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [3]:
!head -1 ../../data/stackoverflow_10mln.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [4]:
!wc -l ../../data/stackoverflow_10mln.tsv

 10000000 ../../data/stackoverflow_10mln.tsv


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлом на входе и на выходе
  - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
  - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
  - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 – *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
  - из текста вопроса надо выкинуть двоеточия и вертикальные палки – в VW это спецсимволы.

In [1]:
!cat preprocess.py

import sys
from tqdm import tqdm

topics = ['javascript', 'java', 'python', 'ruby', 'php',
          'c++', 'c#', 'go', 'scala', 'swift']
topic_set = set(topics)
topic_map = dict(zip(topics, range(1, len(topics) + 1)))

num_corrupted, num_selected = 0, 0
with open(sys.argv[1]) as inp_file, open(sys.argv[2], 'w') as out_file:
    for line in tqdm(inp_file):
        values = line.strip().split('\t')
        if len(values) != 2:
            num_corrupted += 1
            continue
        text, labels = values
        labels = set(labels.split())
        topics_from_list = labels.intersection(topic_set)
        if len(topics_from_list) == 1:
            num_selected += 1
            out_file.write('{} | {}\n'.format(str(topic_map[list(topics_from_list)[0]]), 
                                              text.strip().replace(':', '').replace('|', '')))
print("{} lines selected, {} lines corrupted.".format(num_selected, num_corrupted))



In [36]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк

In [1]:
!python preprocess.py stackoverflow_10mln.tsv stackoverflow.vw

10000000it [01:23, 119062.63it/s]
4389054 lines selected, 15 lines corrupted.


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train_part.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов на для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [19]:
%%time
!split -l 1463018 stackoverflow.vw stackoverflow_

CPU times: user 2.3 s, sys: 821 ms, total: 3.12 s
Wall time: 2min 37s


In [20]:
!mv stackoverflow_aa stackoverflow_train.vw
!mv stackoverflow_ab stackoverflow_valid.vw
!mv stackoverflow_ac stackoverflow_test.vw

In [22]:
!wc -l stackoverflow_*.vw

 1463018 stackoverflow_test.vw
 1463018 stackoverflow_train.vw
 1463018 stackoverflow_valid.vw
 4389054 total


In [31]:
%%time
!cut -f 1 -d ' ' stackoverflow_valid.vw > stackoverflow_valid_labels.txt
!cut -f 1 -d ' ' stackoverflow_test.vw > stackoverflow_test_labels.txt

CPU times: user 686 ms, sys: 244 ms, total: 930 ms
Wall time: 36.5 s


Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [23]:
def train_vw_model(train_vw_file, model_filename, num_classes=10,
                   ngram=1, bit_precision=28, passes=1,
                   seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw --oaa {num_classes} {train_vw_file} ' + 
                       '-f {model_filename} -b {bit_precision} --random_seed {seed}').format(
                       num_classes=num_classes, train_vw_file=train_vw_file, 
                       model_filename=model_filename, bit_precision=bit_precision, seed=seed)
    if ngram > 1:
         vw_call_string += ' --ngram={}'.format(ngram)
            
    if passes > 1:
         vw_call_string += ' -k --passes={} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
    if quiet:
        vw_call_string += ' --quiet'
    
    
    print(vw_call_string) 
    res = os.system(vw_call_string)
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 2))
          if not res else 'Failed.')

In [24]:
def test_vw_model(model_filename, test_vw_file, prediction_filename,
                  true_labels, seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw -t -i {model_filename} {test_vw_file} ' + 
                       '-p {prediction_filename} --random_seed {seed}').format(
                       model_filename=model_filename, test_vw_file=test_vw_file, 
                       prediction_filename=prediction_filename, seed=seed)
    if quiet:
        vw_call_string += ' --quiet'
        
    print(vw_call_string) 
    res = os.system(vw_call_string)
    
    if not res: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("Accuracy: {}%. Elapsed: {} sec.".format(
            round(100 * accuracy_score(true_labels, vw_pred), 2), 
            round(time() - init_time, 2)))
    else:
        print('Failed.')

In [34]:
y_valid = np.loadtxt('stackoverflow_valid_labels.txt')
y_test = np.loadtxt('stackoverflow_test_labels.txt')

In [37]:
for i, (ngram, passes) in tqdm(enumerate(itertools.product([1,2,3], 
                                                      [1,3,5]))):
    train_vw_model('stackoverflow_train.vw', 
                   'vw_model{}_part.vw'.format(i), 
                   ngram=ngram, passes=passes,
                   num_classes=10, bit_precision=28, 
                   seed=17, quiet=True)
    test_vw_model(model_filename='vw_model{}_part.vw'.format(i), 
              test_vw_file='stackoverflow_valid.vw', 
              prediction_filename='vw_valid_pred{}.csv'.format(i),
              true_labels=y_valid, seed=17)

0it [00:00, ?it/s]

vw --oaa 10 stackoverflow_train.vw -f vw_model0_part.vw -b 28 --random_seed 17 --quiet
Success. Elapsed: 34.93 sec.
vw -t -i vw_model0_part.vw stackoverflow_valid.vw -p vw_valid_pred0.csv --random_seed 17 --quiet


1it [01:03, 63.20s/it]

Accuracy: 91.51%. Elapsed: 28.26 sec.
vw --oaa 10 stackoverflow_train.vw -f vw_model1_part.vw -b 28 --random_seed 17 -k --passes=3 --cache_file vw_model1_part.cache --quiet
Success. Elapsed: 114.92 sec.
vw -t -i vw_model1_part.vw stackoverflow_valid.vw -p vw_valid_pred1.csv --random_seed 17 --quiet


2it [03:23, 86.27s/it]

...

Accuracy: 92.85%. Elapsed: 76.28 sec.
vw --oaa 10 stackoverflow_train.vw -f vw_model7_part.vw -b 28 --random_seed 17 --ngram=3 -k --passes=3 --cache_file vw_model7_part.cache --quiet
Success. Elapsed: 548.67 sec.
vw -t -i vw_model7_part.vw stackoverflow_valid.vw -p vw_valid_pred7.csv --random_seed 17 --quiet


8it [34:04, 357.82s/it]

Accuracy: 92.61%. Elapsed: 97.56 sec.
vw --oaa 10 stackoverflow_train.vw -f vw_model8_part.vw -b 28 --random_seed 17 --ngram=3 -k --passes=5 --cache_file vw_model8_part.cache --quiet
Success. Elapsed: 717.71 sec.
vw -t -i vw_model8_part.vw stackoverflow_valid.vw -p vw_valid_pred8.csv --random_seed 17 --quiet


9it [47:19, 489.18s/it]

Accuracy: 92.6%. Elapsed: 77.97 sec.


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 3 прохода по выборке

<font color='red'> Ответ:</font> Лучше всего сработала модель с биграммами и одним проходом по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [39]:
test_vw_model(model_filename='vw_model3_part.vw', 
              test_vw_file='stackoverflow_test.vw', 
              prediction_filename='vw_test_pred3.csv',
              true_labels=y_test, seed=17)

vw -t -i vw_model3_part.vw stackoverflow_test.vw -p vw_test_pred3.csv --random_seed 17 --quiet
Accuracy: 93.11%. Elapsed: 47.2 sec.


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках?
- На тестовой ниже примерно на 1%
- На тестовой ниже примерно на 1%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

<font color='red'> Ответ:</font> Результаты почти одинаковы, что не удивительно, мы поделили исходную выборку на 3 части, обучались на одной из них, значит, на двух других результаты должны быть примерно равными.

In [50]:
%%time
!cp stackoverflow_train.vw stackoverflow_train_valid.vw
!cat stackoverflow_valid.vw >> stackoverflow_train_valid.vw

CPU times: user 309 ms, sys: 122 ms, total: 431 ms
Wall time: 14.2 s


Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [51]:
train_vw_model('stackoverflow_train_valid.vw', 
                   'vw_model10.vw', 
                   ngram=2, passes=1,
                   num_classes=10, bit_precision=28, 
                   seed=17, quiet=True)

vw --oaa 10 stackoverflow_train_valid.vw -f vw_model10.vw -b 28 --random_seed 17 --ngram=2 --quiet
Success. Elapsed: 155.63 sec.


In [52]:
test_vw_model(model_filename='vw_model10.vw', 
              test_vw_file='stackoverflow_test.vw', 
              prediction_filename='vw_test_pred10.csv',
              true_labels=y_test, seed=17)

vw -t -i vw_model10.vw stackoverflow_test.vw -p vw_test_pred10.csv --random_seed 17 --quiet
Accuracy: 93.52%. Elapsed: 50.14 sec.


<font color='red'> Вопрос 3.</font> На сколько повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

<font color='red'> Ответ:</font> Добавление данных помогло, доля правильных ответов возросла на  0.4%. Кстати, результат дискуссионный – стоит ли в реальном приложении мучительно настраивать параметры модели, или хватит 91.5% верных ответов простой модели (униграммы и один проход по выборке). Пожалуй, в данном случае прогнозирования тегов вопросов оно того не стоит. Хотя узнали мы это только понастраивая параметры, заранее не могли знать. 

<font color='red'> Критика </font> данного решения:
- не использовалась обертка `sklearn` для Vowpal Wabbit
- не использовалась библиотека `hyperopt` для настройки параметров
- лучше результаты обучения моделей писать в лог-файл, а не печатать
- если использовать shell-команды, можно обрабатывать данные быстрее, чем скриптом на `Python`

Впрочем, как quick&dirty решение вполне пойдет. Учитывая объем данных, увлекаться настройкой параметров тут не стоит.